In [ ]:
%pip install python-dotenv

In [ ]:
import json
import requests
import pytz
from datetime import datetime, timezone

In [ ]:
# set the Sonarr API URL and key
sonarr_url = 'http://192.168.4.2:8989/api/'
sonarr_key = 'abc123'

# set the API endpoints
history_endpoint = 'history?sortKey=date&sortDir=desc&pageSize=5&page=1&filterKey=eventType&filterValue=3'
calendar_endpoint = 'calendar'

In [ ]:
def time_since(timestamp):
    """Converts an ISO 8601 timestamp to a human-readable string like "15 minutes ago"."""
    now = datetime.now(timezone.utc)
    ts = datetime.strptime(timestamp[:-1], '%Y-%m-%dT%H:%M:%S.%f')
    ts = ts.replace(tzinfo=pytz.UTC)
    delta = now - ts
    
    if delta.days > 0:
        if delta.days == 1:
            return "1 day ago"
        else:
            return "{} days ago".format(delta.days)
    else:
        seconds = delta.seconds
        if seconds < 60:
            return "just now"
        elif seconds < 120:
            return "1 minute ago"
        elif seconds < 3600:
            return "{} minutes ago".format(int(seconds / 60))
        elif seconds < 7200:
            return "1 hour ago"
        else:
            return "{} hours ago".format(int(seconds / 3600))


In [ ]:
def recently_added_episodes(index=""):
    # make the API request
    response = requests.get(sonarr_url + history_endpoint, headers={'X-Api-Key': sonarr_key})

    # check if the request was successful
    if response.status_code == 200:
        # parse the JSON response
        data = response.json()
        # get the list of episodes
        episodes = data['records']
        # create a list to hold the episode details
        episode_details = []
        # loop through the episodes and add their details to the list
        for episode in episodes:
            episode_detail = {
                'show_title': episode['series']['title'],
                'episode_title': episode['episode']['title'],
                'season_number': episode['episode']['seasonNumber'],
                'episode_number': episode['episode']['episodeNumber'],
                'time_since_added': time_since(episode['date'])
            }
            episode_details.append(episode_detail)
        return episode_details
    else:
        # print an error message if the request was unsuccessful
        print('Error: {}'.format(response.text))

In [ ]:
def upcoming_episodes(index=""):
    # make the API request
    response = requests.get(sonarr_url + calendar_endpoint, headers={'X-Api-Key': sonarr_key})

    # check if the request was successful
    if response.status_code == 200:
        # parse the JSON response into a dictionary
        data = response.json()
        # get the list of episodes
        episodes = data
        # create a list to store the episode details
        episode_details = []
        for episode in episodes:
            # create a dictionary to store the episode details
            episode_dict = {}
            episode_dict['show'] = episode['series']['title']
            episode_dict['title'] = episode['title']
            episode_dict['episode'] = 'S{}E{}'.format(episode['seasonNumber'], episode['episodeNumber'])
            episode_dict['air_date'] = episode['airDateUtc']
            episode_details.append(episode_dict)
        return episode_details
    else:
        # raise an exception if the request was unsuccessful
        raise Exception('Error: {}'.format(response.text))


In [ ]:
print(json.dumps(recently_added_episodes(), indent=2))

In [ ]:
print(json.dumps(upcoming_episodes(), indent=2))

In [ ]:
!jupyter nbconvert --to python sonarr_recent.ipynb